In [193]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization
from tensorflow.keras.models import Model
import cv2
import os
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError

In [268]:
# mnist.load_data() returns 2 tuples having 2 values each for X and y respectively
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [269]:
# To have pixel values between 0 and 1
X_train = X_train / 255.0
X_test = X_test / 255.0

In [270]:
# One hot encoding
num_classes = 10
y_train_one_hot = to_categorical(y_train, num_classes)
y_test_one_hot = to_categorical(y_test, num_classes)

# ANN for MNIST dataset

In [6]:
# Generalized functional API function to have variable number of dense layers and variable number of neurons
def func_api(input_shape, no_of_dense,neurons):
    inputs = keras.Input(shape=input_shape)
    x = layers.Flatten()(inputs)
    for i in range(0,no_of_dense):
        x = keras.layers.Dense(neurons, activation="relu")(x)
    outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [7]:
def func_api_with_skip(input_shape, no_of_dense, neurons,layer_no_1,layer_no_2):
    inputs = keras.Input(shape=input_shape)
    x = layers.Flatten()(inputs)
    dense_outputs = []
    for i in range(0, no_of_dense):
        x = keras.layers.Dense(neurons, activation="relu")(x)
        dense_outputs.append(x)
    for i, output in enumerate(dense_outputs):
      print(f"Dense Output {i+1}: {output}")
    skip_connection = layers.add([dense_outputs[layer_no_1], dense_outputs[layer_no_2]])
    outputs = keras.layers.Dense(num_classes, activation="softmax")(skip_connection)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [265]:
def compilee(model):
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  model.summary()

In [266]:
def print_eval(model):
  y_pred = model.predict(X_test)
  y_pred_classes = np.argmax(y_pred, axis=1)
  y_true_classes = np.argmax(y_test_one_hot, axis=1)
  conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
  print("Confusion Matrix:")
  print(conf_matrix)
  class_report = classification_report(y_true_classes, y_pred_classes)
  print("\nClassification Report:")
  print(class_report)
  test_accuracy = np.sum(y_pred_classes == y_true_classes) / len(y_true_classes)
  print("\nTest Accuracy:", test_accuracy)

# ANN Without Skip Connection

In [10]:
# Calling Generalized functional API function to create a model having 1 dense layer with 32 neurons
model = func_api((28,28), 0,32)

In [11]:
compilee(model)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 10)                7850      
                                                                 
Total params: 7850 (30.66 KB)
Trainable params: 7850 (30.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.fit(X_train, y_train_one_hot, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5123 - accuracy: 0.8670 - val_loss: 0.3161 - val_accuracy: 0.9122
Epoch 2/5
1500/1500 [==============================] - 1s 935us/step - loss: 0.3178 - accuracy: 0.9119 - val_loss: 0.2887 - val_accuracy: 0.9195
Epoch 3/5
1500/1500 [==============================] - 1s 933us/step - loss: 0.2933 - accuracy: 0.9190 - val_loss: 0.2788 - val_accuracy: 0.9212
Epoch 4/5
1500/1500 [==============================] - 1s 939us/step - loss: 0.2812 - accuracy: 0.9207 - val_loss: 0.2719 - val_accuracy: 0.9266
Epoch 5/5
1500/1500 [==============================] - 1s 945us/step - loss: 0.2736 - accuracy: 0.9234 - val_loss: 0.2648 - val_accuracy: 0.9266


In [13]:
print_eval(model)

313/313 [==============================] - 0s 667us/step
Confusion Matrix:
[[ 955    0    1    1    0    7   10    3    3    0]
 [   0 1111    3    2    0    2    4    2   11    0]
 [   3    6  932   16    7    5   11   10   37    5]
 [   2    0   23  917    0   22    2   11   24    9]
 [   1    1    7    0  912    0   10    3    8   40]
 [   8    3    6   36    6  766   17    6   36    8]
 [   9    3    7    1    7   12  915    2    2    0]
 [   1    5   24    5    7    0    0  947    2   37]
 [   4    7    7   16    9   20   10   11  876   14]
 [   9    5    2    9   21    8    0   17    8  930]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       980
           1       0.97      0.98      0.98      1135
           2       0.92      0.90      0.91      1032
           3       0.91      0.91      0.91      1010
           4       0.94      0.93      0.93       982
           5       0.91      0.86      0.88  

In [14]:
# Calling Generalized functional API function to create a model having 3 dense layer with 32 neurons
# Also compiling and fitting the model
model = func_api((28,28), 2,32)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, batch_size=32, validation_split=0.2)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                25120     
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 10)                330       
                                                                 
Total params: 26506 (103.54 KB)
Trainable params: 26506 (103.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
1500/1500 [

In [15]:
print_eval(model)

313/313 [==============================] - 0s 657us/step
Confusion Matrix:
[[ 969    0    2    2    0    1    3    2    1    0]
 [   0 1122    1    2    0    1    5    0    4    0]
 [   3    2  994   12    1    1    2    5   12    0]
 [   0    1    3  986    1    6    1    6    5    1]
 [   2    0   12    2  934    0    7    5    5   15]
 [   3    0    0   20    3  845   11    0    8    2]
 [   5    3    4    1    1    9  930    1    4    0]
 [   1    9   14   18    0    1    0  971    1   13]
 [   2    2    4   31    3    2    6    5  919    0]
 [   6    6    2   21   17    6    1   15   20  915]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       980
           1       0.98      0.99      0.98      1135
           2       0.96      0.96      0.96      1032
           3       0.90      0.98      0.94      1010
           4       0.97      0.95      0.96       982
           5       0.97      0.95      0.96  

In [16]:
# Calling Generalized functional API function to create a model having 5 dense layer with 32 neurons
# Also compiling and fitting the model
model = func_api((28,28), 4,32)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, batch_size=32, validation_split=0.2)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 32)                25120     
                                                                 
 dense_5 (Dense)             (None, 32)                1056      
                                                                 
 dense_6 (Dense)             (None, 32)                1056      
                                                                 
 dense_7 (Dense)             (None, 32)                1056      
                                                                 
 dense_8 (Dense)             (None, 10)                330 

In [17]:
print_eval(model)

313/313 [==============================] - 0s 692us/step
Confusion Matrix:
[[ 964    0    1    3    1    3    6    0    1    1]
 [   0 1120    5    4    0    1    3    0    2    0]
 [   6    0 1006    8    2    2    4    2    2    0]
 [   0    0   18  977    0    4    0    4    4    3]
 [   1    0    6    0  950    0    4    3    0   18]
 [   2    0    3   18    1  852    9    1    2    4]
 [  10    3    2    2    4    4  932    0    1    0]
 [   0    4   21    6    2    1    0  979    2   13]
 [   7    0    9   20    6   10    4    4  908    6]
 [   7    2    0   12   12    6    0    7    2  961]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.94      0.97      0.96      1032
           3       0.93      0.97      0.95      1010
           4       0.97      0.97      0.97       982
           5       0.96      0.96      0.96  

In [18]:
# Calling Generalized functional API function to create a model having 7 dense layer with 32 neurons
# Also compiling and fitting the model
model = func_api((28,28), 6,32)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, batch_size=32, validation_split=0.2)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 32)                25120     
                                                                 
 dense_10 (Dense)            (None, 32)                1056      
                                                                 
 dense_11 (Dense)            (None, 32)                1056      
                                                                 
 dense_12 (Dense)            (None, 32)                1056      
                                                                 
 dense_13 (Dense)            (None, 32)                1056

In [19]:
print_eval(model)

313/313 [==============================] - 0s 724us/step
Confusion Matrix:
[[ 968    1    0    1    1    3    3    1    2    0]
 [   0 1120    3    0    0    3    6    0    3    0]
 [   5    1  994    4    4    1    6    3   12    2]
 [   0    0   10  920    0   39    0    7   31    3]
 [   0    1    4    0  954    0    7    1    4   11]
 [   8    0    3    5    2  846    8    0   18    2]
 [   6    2    1    0    8    8  928    0    5    0]
 [   2    6   18    8    3    2    0  962    8   19]
 [   9    0    8    0    4    6    2    2  942    1]
 [   6    4    1    3   16   12    1    5   38  923]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.95      0.96      0.96      1032
           3       0.98      0.91      0.94      1010
           4       0.96      0.97      0.97       982
           5       0.92      0.95      0.93  

In [20]:
# Observation keeping the number of neurons same I tried to vary the number of dense layers first it showed improvement in the results till 3 layers, but after that when dense layers
# were increased there was no improvement, instead model was somewhat overfitting.

# ANN with Skip Connection

In [21]:
model_skip = func_api_with_skip((28,28),3,32,0,2)

Dense Output 1: KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='dense_16/Relu:0', description="created by layer 'dense_16'")
Dense Output 2: KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='dense_17/Relu:0', description="created by layer 'dense_17'")
Dense Output 3: KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='dense_18/Relu:0', description="created by layer 'dense_18'")


In [22]:
compilee(model_skip)
model_skip.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 28, 28)]             0         []                            
                                                                                                  
 flatten_4 (Flatten)         (None, 784)                  0         ['input_5[0][0]']             
                                                                                                  
 dense_16 (Dense)            (None, 32)                   25120     ['flatten_4[0][0]']           
                                                                                                  
 dense_17 (Dense)            (None, 32)                   1056      ['dense_16[0][0]']            
                                                                                            

In [23]:
print_eval(model_skip)

313/313 [==============================] - 0s 713us/step
Confusion Matrix:
[[ 965    0    1    1    0    4    3    1    2    3]
 [   0 1120    3    6    0    1    1    0    4    0]
 [   3    2  998   13    4    0    2    6    3    1]
 [   0    0    4  995    1    2    0    3    4    1]
 [   1    2    4    1  936    0    6    4    3   25]
 [   3    1    1   27    0  849    6    0    3    2]
 [   6    3    3    1    4   11  925    0    5    0]
 [   2    9   11    8    2    1    0  978    1   16]
 [   3    2    5   29    3   12    2    4  909    5]
 [   5    5    0    9   10    3    1    6    1  969]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       980
           1       0.98      0.99      0.98      1135
           2       0.97      0.97      0.97      1032
           3       0.91      0.99      0.95      1010
           4       0.97      0.95      0.96       982
           5       0.96      0.95      0.96  

In [263]:
def func_api_aux_loss(input_shape, no_of_dense, neurons):
    inputs = keras.Input(shape=input_shape)
    x = layers.Flatten()(inputs)
    for i in range(no_of_dense):
        x = keras.layers.Dense(neurons, activation="relu")(x)
    main_output = keras.layers.Dense(10, activation="softmax")(x)
    aux_output = keras.layers.Dense(10, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=[main_output, aux_output])
    return model


def print_eval_aux(model):
    y_main_pred, y_aux_pred = model.predict(X_test)
    y_main_pred_classes = np.argmax(y_main_pred, axis=1)
    y_true_classes = np.argmax(y_test_one_hot, axis=1)
    print("\nEvaluation for Main Task:")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true_classes, y_main_pred_classes))
    print("\nClassification Report:")
    print(classification_report(y_true_classes, y_main_pred_classes))
    test_accuracy_main = np.sum(y_main_pred_classes == y_true_classes) / len(y_true_classes)
    print("\nTest Accuracy for Main Task:", test_accuracy_main)
    y_aux_pred_classes = np.argmax(y_aux_pred, axis=1)
    print("\nEvaluation for Auxiliary Task:")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true_classes, y_aux_pred_classes))
    print("\nClassification Report:")
    print(classification_report(y_true_classes, y_aux_pred_classes))
    test_accuracy_aux = np.sum(y_aux_pred_classes == y_true_classes) / len(y_true_classes)
    print("\nTest Accuracy for Auxiliary Task:", test_accuracy_aux)





In [271]:
model = func_api_aux_loss((28,28), 3, 32)
compilee(model)
model.fit(X_train, [y_train_one_hot, y_train_one_hot], epochs=5, validation_split=0.2)

print_eval_aux(model)

Model: "model_22"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_26 (InputLayer)       [(None, 28, 28)]             0         []                            
                                                                                                  
 flatten_9 (Flatten)         (None, 784)                  0         ['input_26[0][0]']            
                                                                                                  
 dense_48 (Dense)            (None, 32)                   25120     ['flatten_9[0][0]']           
                                                                                                  
 dense_49 (Dense)            (None, 32)                   1056      ['dense_48[0][0]']            
                                                                                           

# CNN for MNIST dataset

In [24]:
# Generalized CNN function having Functional API structure and variable number of Convo and Dense layers
def cnn_model(input_shape, no_of_convo, no_of_dense, neurons):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, kernel_size=(3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    for k in range(no_of_convo):
        x = Conv2D(32, kernel_size=(3, 3), padding='same')(x)  
        x = BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    for k in range(no_of_dense):
        x = Dense(neurons, activation='relu')(x)
        x = BatchNormalization()(x)
    outputs = Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [25]:
model = cnn_model((28, 28,1),-1,0,32)

In [26]:
compilee(model)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 32)        128       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 flatten_5 (Flatten)         (None, 6272)              0   

In [27]:
model.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Epoch 1/5
1500/1500 [==============================] - 16s 11ms/step - loss: 0.1783 - accuracy: 0.9465 - val_loss: 0.0855 - val_accuracy: 0.9750
Epoch 2/5
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0755 - accuracy: 0.9766 - val_loss: 0.0721 - val_accuracy: 0.9798
Epoch 3/5
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0561 - accuracy: 0.9829 - val_loss: 0.0906 - val_accuracy: 0.9756
Epoch 4/5
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0439 - accuracy: 0.9858 - val_loss: 0.0768 - val_accuracy: 0.9783
Epoch 5/5
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0348 - accuracy: 0.9887 - val_loss: 0.0780 - val_accuracy: 0.9789


In [28]:
print_eval(model)

313/313 [==============================] - 1s 4ms/step
Confusion Matrix:
[[ 975    0    1    1    0    0    1    0    2    0]
 [   0 1122    3    0    1    0    3    1    5    0]
 [   2    1 1011    1    1    0    1    8    6    1]
 [   0    0    2  994    0    1    0    5    7    1]
 [   2    0    1    0  956    0    4    3    2   14]
 [   3    1    2   13    0  852    6    0   11    4]
 [   8    1    0    1    1    2  940    0    5    0]
 [   1    3    5    1    2    0    0 1014    0    2]
 [   5    0    2    0    0    1    1    4  958    3]
 [   3    4    1    2    3    1    0    7    3  985]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.98      0.98      0.98      1010
           4       0.99      0.97      0.98       982
           5       0.99      0.96      0.97    

In [29]:
model = cnn_model((28, 28, 1),1,2,32)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 batch_normalization_1 (Bat  (None, 28, 28, 32)        128       
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 32)        9248

In [30]:
print_eval(model)

313/313 [==============================] - 1s 4ms/step
Confusion Matrix:
[[ 974    0    1    0    0    1    3    1    0    0]
 [   0 1130    2    1    0    0    1    0    1    0]
 [   1    1 1022    0    1    0    0    6    0    1]
 [   1    0    6  998    0    4    0    1    0    0]
 [   0    2    2    0  938    0    4    5    0   31]
 [   1    0    1   11    0  878    1    0    0    0]
 [   4    2    5    0    1    2  944    0    0    0]
 [   0    6    9    0    0    0    0 1011    1    1]
 [   2    0    3    3    4    6    6    2  940    8]
 [   1    0    1    1    1    3    0    4    0  998]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.97      0.99      0.98      1032
           3       0.98      0.99      0.99      1010
           4       0.99      0.96      0.97       982
           5       0.98      0.98      0.98    

In [31]:
model = cnn_model((28, 28, 1),2,2,32)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 batch_normalization_5 (Bat  (None, 28, 28, 32)        128       
 chNormalization)                                                
                                                                 
 activation_3 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 32)        9248

print_eval(model)

In [33]:
model = cnn_model((28, 28, 1),2,4,32)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 batch_normalization_10 (Ba  (None, 28, 28, 32)        128       
 tchNormalization)                                               
                                                                 
 activation_6 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 32)        9248

In [34]:
print_eval(model)

313/313 [==============================] - 2s 5ms/step
Confusion Matrix:
[[ 974    1    1    0    0    0    0    2    1    1]
 [   0 1129    0    5    0    0    1    0    0    0]
 [   0    1 1026    3    0    0    1    1    0    0]
 [   0    1    0 1008    0    1    0    0    0    0]
 [   0    4    0    0  969    0    1    1    1    6]
 [   2    0    0    7    0  879    3    1    0    0]
 [  10    3    1    0    0    2  942    0    0    0]
 [   0    5   11    2    0    0    0 1010    0    0]
 [   2    1    1   13    0    1    1    0  954    1]
 [   2    3    1    7    3    7    0    5    5  976]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.98      0.99      0.99      1135
           2       0.99      0.99      0.99      1032
           3       0.96      1.00      0.98      1010
           4       1.00      0.99      0.99       982
           5       0.99      0.99      0.99    

# CNN with skip connection for MNIST

In [35]:
def cnn_model_with_skip(input_shape, no_of_convo, no_of_dense, neurons, convo_no_1, convo_no_2, layer_no_1, layer_no_2, num_classes=10):
    inputs = Input(shape=input_shape)
    x = inputs
    conv_output = x
    outputs_convo = []
    for k in range(0,no_of_convo):
        conv_output = tf.keras.layers.Activation("relu")(Conv2D(32, kernel_size=(3, 3), padding='same')(conv_output))
        outputs_convo.append(conv_output)
        print(f"Convo Layer {k+1}: {conv_output}")
    x = layers.add([outputs_convo[convo_no_1], outputs_convo[convo_no_2]])
    x = Flatten()(x)
    dense_outputs = []
    for i in range(0, no_of_dense):
        x = Dense(neurons, activation="relu")(x)
        dense_outputs.append(x)


    skip_connection = layers.add([dense_outputs[layer_no_1], dense_outputs[layer_no_2]])
    outputs = Dense(10, activation="softmax")(skip_connection)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [36]:
model_cnn_skip = cnn_model_with_skip((28, 28, 1), 2, 3, 32,0,1,1,2)


Convo Layer 1: KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 32), dtype=tf.float32, name=None), name='activation_9/Relu:0', description="created by layer 'activation_9'")
Convo Layer 2: KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 32), dtype=tf.float32, name=None), name='activation_10/Relu:0', description="created by layer 'activation_10'")


In [37]:
compilee(model_cnn_skip)
model_cnn_skip.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d_9 (Conv2D)           (None, 28, 28, 32)           320       ['input_10[0][0]']            
                                                                                                  
 activation_9 (Activation)   (None, 28, 28, 32)           0         ['conv2d_9[0][0]']            
                                                                                                  
 conv2d_10 (Conv2D)          (None, 28, 28, 32)           9248      ['activation_9[0][0]']        
                                                                                            

In [38]:
print_eval(model_cnn_skip)

313/313 [==============================] - 2s 6ms/step
Confusion Matrix:
[[ 970    0    1    0    3    0    3    1    2    0]
 [   0 1135    0    0    0    0    0    0    0    0]
 [   4    3 1013    0    0    0    0    6    6    0]
 [   0    0    1 1005    0    1    0    1    2    0]
 [   0    1    0    0  977    0    0    0    0    4]
 [   0    0    0    9    0  870    5    0    2    6]
 [   3    2    0    0    2    5  944    0    2    0]
 [   1    5    4    1    2    0    0 1010    0    5]
 [   2    2    2    1    2    0    7    2  950    6]
 [   0    3    0    0    7    3    0    0    0  996]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.99      0.98      0.99      1032
           3       0.99      1.00      0.99      1010
           4       0.98      0.99      0.99       982
           5       0.99      0.98      0.98    

In [39]:
model_cnn_skip = cnn_model_with_skip((28, 28, 1), 4, 4, 32,0,2,1,3)
compilee(model_cnn_skip)
model_cnn_skip.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Convo Layer 1: KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 32), dtype=tf.float32, name=None), name='activation_11/Relu:0', description="created by layer 'activation_11'")


Convo Layer 2: KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 32), dtype=tf.float32, name=None), name='activation_12/Relu:0', description="created by layer 'activation_12'")
Convo Layer 3: KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 32), dtype=tf.float32, name=None), name='activation_13/Relu:0', description="created by layer 'activation_13'")
Convo Layer 4: KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 32), dtype=tf.float32, name=None), name='activation_14/Relu:0', description="created by layer 'activation_14'")
Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d_11 (Conv2D)          (None, 28, 28

In [40]:
print_eval(model_cnn_skip)

313/313 [==============================] - 3s 8ms/step
Confusion Matrix:
[[ 971    0    1    1    3    0    0    1    2    1]
 [   0 1126    0    4    0    0    2    2    0    1]
 [   1    3 1010    4    1    0    0    8    5    0]
 [   0    0    2 1007    0    0    0    0    1    0]
 [   0    1    0    0  976    0    0    2    0    3]
 [   0    0    0   20    0  866    1    0    4    1]
 [   2    3    0    2   16    3  924    0    7    1]
 [   0    4    4    5    0    0    0 1008    1    6]
 [   1    1    2    0    0    0    0    2  962    6]
 [   0    2    0    1    8    2    0    3    2  991]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.99      0.98      0.98      1032
           3       0.96      1.00      0.98      1010
           4       0.97      0.99      0.98       982
           5       0.99      0.97      0.98    

In [41]:
# Observations: When the number of dense layers and convo layers both were increased our model performed better, but after a certain number of convo layers and dense layers the
# model's accuracy got reduced a bit and the cause is overfitting.

# ANN For CIFAR-10

In [202]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
# Normalize pixel values to the range [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0
# One-hot encoding
num_classes = 10
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes)

# ANN for Cifar-10

In [43]:
model = func_api((32,32,3), 1,32)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 flatten_11 (Flatten)        (None, 3072)              0         
                                                                 
 dense_41 (Dense)            (None, 32)                98336     
                                                                 
 dense_42 (Dense)            (None, 10)                330       
                                                                 
Total params: 98666 (385.41 KB)
Trainable params: 98666 (385.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
1250/1250 [==============================] - 3s 2ms/step - loss: 2.3047 - accuracy: 0.0998 - val_loss: 2.3027 - val_accuracy: 0.0997
Epoch 2/5

In [44]:
print_eval(model)

313/313 [==============================] - 0s 722us/step
Confusion Matrix:
[[101  32   0   0   0   0 219 187 461   0]
 [115  79   0   0   0   0 121 171 514   0]
 [ 52  28   0   0   0   0 624 187 109   0]
 [ 68  32   0   0   0   0 579 259  62   0]
 [ 40  13   0   0   0   0 726 162  59   0]
 [ 57  20   0   0   0   0 670 213  40   0]
 [ 27  15   0   0   0   0 779 159  20   0]
 [ 82  23   0   0   0   0 580 247  68   0]
 [116  51   0   0   0   0  66 155 612   0]
 [111  74   0   0   0   0  99 147 569   0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.13      0.10      0.11      1000
           1       0.22      0.08      0.12      1000
           2       0.00      0.00      0.00      1000
           3       0.00      0.00      0.00      1000
           4       0.00      0.00      0.00      1000
           5       0.00      0.00      0.00      1000
           6       0.17      0.78      0.29      1000
           7       0.13      0.25    

c:\Users\Harshneel\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Harshneel\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Harshneel\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

In [45]:
model = func_api((32,32,3), 3,32)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 flatten_12 (Flatten)        (None, 3072)              0         
                                                                 
 dense_43 (Dense)            (None, 32)                98336     
                                                                 
 dense_44 (Dense)            (None, 32)                1056      
                                                                 
 dense_45 (Dense)            (None, 32)                1056      
                                                                 
 dense_46 (Dense)            (None, 10)                330       
                                                                 
Total params: 100778 (393.66 KB)
Trainable params: 100778 

In [46]:
print_eval(model)

313/313 [==============================] - 0s 746us/step
Confusion Matrix:
[[480  58  21  22  24   7  34  64 226  64]
 [ 37 564   6  14  21  10  38  40 121 149]
 [110  37 130  78 163  43 207 141  67  24]
 [ 51  44  38 223  57 111 204 130  55  87]
 [ 67  27  82  74 300  17 201 164  37  31]
 [ 30  19  42 155  80 238 175 154  67  40]
 [ 12  22  40  90 115  50 562  48  12  49]
 [ 67  42  20  48  91  27  69 516  31  89]
 [ 97 105   7  37   8  21  14  17 604  90]
 [ 31 275   1  27   5  10  40  59 115 437]]

Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.48      0.48      1000
           1       0.47      0.56      0.51      1000
           2       0.34      0.13      0.19      1000
           3       0.29      0.22      0.25      1000
           4       0.35      0.30      0.32      1000
           5       0.45      0.24      0.31      1000
           6       0.36      0.56      0.44      1000
           7       0.39      0.52    

In [47]:
model = func_api((32,32,3), 5,32)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 flatten_13 (Flatten)        (None, 3072)              0         
                                                                 
 dense_47 (Dense)            (None, 32)                98336     
                                                                 
 dense_48 (Dense)            (None, 32)                1056      
                                                                 
 dense_49 (Dense)            (None, 32)                1056      
                                                                 
 dense_50 (Dense)            (None, 32)                1056      
                                                                 
 dense_51 (Dense)            (None, 32)                105

In [48]:
print_eval(model)

313/313 [==============================] - 0s 753us/step
Confusion Matrix:
[[397  61  20  48  12  25  30  48 238 121]
 [ 24 461  10  34  13  49  43  29  88 249]
 [139  48  51  91 146 112 208 113  48  44]
 [ 35  81  36 170  64 264 156  61  45  88]
 [ 44  30  46  78 240  87 253 128  48  46]
 [ 32  56  31 132  78 393 126  64  44  44]
 [  8  42  23 108  88 117 504  44  11  55]
 [ 49  33  31  72  97 101  96 345  39 137]
 [101 104  10  21   3  55   4  12 472 218]
 [ 22 173   6  29   3  22  37  41 104 563]]

Classification Report:
              precision    recall  f1-score   support

           0       0.47      0.40      0.43      1000
           1       0.42      0.46      0.44      1000
           2       0.19      0.05      0.08      1000
           3       0.22      0.17      0.19      1000
           4       0.32      0.24      0.28      1000
           5       0.32      0.39      0.35      1000
           6       0.35      0.50      0.41      1000
           7       0.39      0.34    

In [49]:
model = func_api((32,32,3), 5,64)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 flatten_14 (Flatten)        (None, 3072)              0         
                                                                 
 dense_53 (Dense)            (None, 64)                196672    
                                                                 
 dense_54 (Dense)            (None, 64)                4160      
                                                                 
 dense_55 (Dense)            (None, 64)                4160      
                                                                 
 dense_56 (Dense)            (None, 64)                4160      
                                                                 
 dense_57 (Dense)            (None, 64)                416

In [50]:
print_eval(model)

313/313 [==============================] - 0s 772us/step
Confusion Matrix:
[[487  45 151  34  38  36  32  47 108  22]
 [ 54 562  31  34  23  43  41  57  58  97]
 [ 74  19 329  46 203  84 157  69  13   6]
 [ 18  17 115 199  90 188 266  73  16  18]
 [ 47  11 172  48 421  46 172  65   8  10]
 [ 13  11 134 131  99 352 164  78  12   6]
 [  3  20 108  59 131  36 603  29   5   6]
 [ 49  19  78  37 172  92  70 455   7  21]
 [182  79  67  32  19  58  32  24 454  53]
 [ 67 196  36  43  22  33  58  99  69 377]]

Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.49      0.49      1000
           1       0.57      0.56      0.57      1000
           2       0.27      0.33      0.30      1000
           3       0.30      0.20      0.24      1000
           4       0.35      0.42      0.38      1000
           5       0.36      0.35      0.36      1000
           6       0.38      0.60      0.46      1000
           7       0.46      0.46    

In [51]:
model = func_api((32,32,3), 3,128)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 flatten_15 (Flatten)        (None, 3072)              0         
                                                                 
 dense_59 (Dense)            (None, 128)               393344    
                                                                 
 dense_60 (Dense)            (None, 128)               16512     
                                                                 
 dense_61 (Dense)            (None, 128)               16512     
                                                                 
 dense_62 (Dense)            (None, 10)                1290      
                                                                 
Total params: 427658 (1.63 MB)
Trainable params: 427658 (1

In [52]:
print_eval(model)

313/313 [==============================] - 0s 917us/step
Confusion Matrix:
[[544  34   9  18   9  37  11   7 233  98]
 [ 52 503   5  10   6  30   5  10 127 252]
 [165  52 150  73  96 227  58  43  83  53]
 [ 82  71  23 161  11 370  39  18  97 128]
 [ 95  39  81  62 239 232  72  52  51  77]
 [ 50  53  24 109  13 511  28  32  90  90]
 [ 42  71  33  98  68 221 326   7  50  84]
 [ 85  38  29  37  44 192  17 322  56 180]
 [101  50   0   9   6  25   1   8 709  91]
 [ 57 146   1  10   5  21   5  15 127 613]]

Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.54      0.48      1000
           1       0.48      0.50      0.49      1000
           2       0.42      0.15      0.22      1000
           3       0.27      0.16      0.20      1000
           4       0.48      0.24      0.32      1000
           5       0.27      0.51      0.36      1000
           6       0.58      0.33      0.42      1000
           7       0.63      0.32    

In [53]:
model = func_api((32,32,3), 5,128)
compilee(model)
model.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 flatten_16 (Flatten)        (None, 3072)              0         
                                                                 
 dense_63 (Dense)            (None, 128)               393344    
                                                                 
 dense_64 (Dense)            (None, 128)               16512     
                                                                 
 dense_65 (Dense)            (None, 128)               16512     
                                                                 
 dense_66 (Dense)            (None, 128)               16512     
                                                                 
 dense_67 (Dense)            (None, 128)               165

In [54]:
print_eval(model)

313/313 [==============================] - 0s 1ms/step
Confusion Matrix:
[[516  42  59  32  35  17  28  37 178  56]
 [ 43 592   8  39  23  30  26  20  93 126]
 [ 84  26 221  85 245  90 126  66  38  19]
 [ 40  28  71 271  82 241 142  41  40  44]
 [ 53  10 103  60 453  63 141  69  33  15]
 [ 23  14  77 163  98 385 116  52  53  19]
 [  8  16  69 106 177  69 507  18  11  19]
 [ 43  21  56  56 167  99  46 417  27  68]
 [ 94  70  14  40  23  20   4  13 615 107]
 [ 42 213  13  44  15  27  26  35  75 510]]

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.52      0.53      1000
           1       0.57      0.59      0.58      1000
           2       0.32      0.22      0.26      1000
           3       0.30      0.27      0.29      1000
           4       0.34      0.45      0.39      1000
           5       0.37      0.39      0.38      1000
           6       0.44      0.51      0.47      1000
           7       0.54      0.42      

# ANN with skip connection Cifar-10

In [55]:
model_skip = func_api_with_skip((32,32,3),3,32,0,2)
compilee(model_skip)
model_skip.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Dense Output 1: KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='dense_69/Relu:0', description="created by layer 'dense_69'")


Dense Output 2: KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='dense_70/Relu:0', description="created by layer 'dense_70'")
Dense Output 3: KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='dense_71/Relu:0', description="created by layer 'dense_71'")
Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_18 (InputLayer)       [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 flatten_17 (Flatten)        (None, 3072)                 0         ['input_18[0][0]']            
                                                                                                  
 dense_69 (Dense)            (None, 32)                   9

In [56]:
print_eval(model)

313/313 [==============================] - 0s 1ms/step
Confusion Matrix:
[[516  42  59  32  35  17  28  37 178  56]
 [ 43 592   8  39  23  30  26  20  93 126]
 [ 84  26 221  85 245  90 126  66  38  19]
 [ 40  28  71 271  82 241 142  41  40  44]
 [ 53  10 103  60 453  63 141  69  33  15]
 [ 23  14  77 163  98 385 116  52  53  19]
 [  8  16  69 106 177  69 507  18  11  19]
 [ 43  21  56  56 167  99  46 417  27  68]
 [ 94  70  14  40  23  20   4  13 615 107]
 [ 42 213  13  44  15  27  26  35  75 510]]

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.52      0.53      1000
           1       0.57      0.59      0.58      1000
           2       0.32      0.22      0.26      1000
           3       0.30      0.27      0.29      1000
           4       0.34      0.45      0.39      1000
           5       0.37      0.39      0.38      1000
           6       0.44      0.51      0.47      1000
           7       0.54      0.42      

# CNN for Cifar-10

In [57]:
mode_cnn = cnn_model((32, 32, 3),1,2,32)
compilee(mode_cnn)
mode_cnn.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_17 (Ba  (None, 32, 32, 32)        128       
 tchNormalization)                                               
                                                                 
 activation_15 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 16, 16, 32)        924

In [58]:
print_eval(mode_cnn)

313/313 [==============================] - 2s 7ms/step
Confusion Matrix:
[[723  13  81  21  11  18   1  45  43  44]
 [ 44 675  12  21   2  10   2  43  27 164]
 [ 76   1 559  48  40  70   7 184   7   8]
 [ 21   1  71 436  29 246   3 170   8  15]
 [ 25   1 124  71 400  51   5 315   7   1]
 [ 11   0  36 109  14 633   1 186   4   6]
 [  7   4 102 202  69  72 416 115   4   9]
 [ 10   0  16   8   9  26   0 925   1   5]
 [135  29  18  15   3  12   0  23 720  45]
 [ 31  47  10  12   1  15   0  70  20 794]]

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.72      0.69      1000
           1       0.88      0.68      0.76      1000
           2       0.54      0.56      0.55      1000
           3       0.46      0.44      0.45      1000
           4       0.69      0.40      0.51      1000
           5       0.55      0.63      0.59      1000
           6       0.96      0.42      0.58      1000
           7       0.45      0.93      

In [59]:
mode_cnn = cnn_model((32, 32, 3),3,4,32)
compilee(mode_cnn)
mode_cnn.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_17 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_21 (Ba  (None, 32, 32, 32)        128       
 tchNormalization)                                               
                                                                 
 activation_17 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_18 (Conv2D)          (None, 16, 16, 32)        924

In [60]:
print_eval(mode_cnn)

313/313 [==============================] - 3s 9ms/step
Confusion Matrix:
[[609   6 128   2  91   1  49   3 102   9]
 [ 37 692  14   8   3   3 105   1 108  29]
 [ 42   1 583   9  74  21 248   6  15   1]
 [ 11   2 190 196  76  80 408   3  32   2]
 [  9   3 125  10 426   5 408   3  11   0]
 [  9   0 203  80  68 333 277   5  25   0]
 [  3   0  35   7   4   2 945   0   4   0]
 [  6   3 159   9 214  51 199 349   9   1]
 [ 47  12  33   1  14   2  54   0 834   3]
 [ 31  71  49  13  12   7 206   3 135 473]]

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.61      0.68      1000
           1       0.88      0.69      0.77      1000
           2       0.38      0.58      0.46      1000
           3       0.59      0.20      0.29      1000
           4       0.43      0.43      0.43      1000
           5       0.66      0.33      0.44      1000
           6       0.33      0.94      0.48      1000
           7       0.94      0.35      

In [61]:
mode_cnn = cnn_model((32, 32, 3),3,4,32)
compilee(mode_cnn)
mode_cnn.fit(X_train, y_train_one_hot, epochs=10, validation_split=0.2)

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_29 (Ba  (None, 32, 32, 32)        128       
 tchNormalization)                                               
                                                                 
 activation_21 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 16, 16, 32)        924

In [62]:
print_eval(mode_cnn)

313/313 [==============================] - 3s 10ms/step
Confusion Matrix:
[[640  17  31  16  80   2   3  10 156  45]
 [ 30 802   4   1   2   2  10   0  37 112]
 [ 69  10 579  25 102  22  91  40  40  22]
 [ 37  16 128 318 125  80 119  30  92  55]
 [ 25   1  85  28 671  15  97  40  22  16]
 [ 20  13 129 132 113 360  75  66  55  37]
 [ 15   7  56  22  32   6 811   2  32  17]
 [ 45   9  52  25  80  31  17 667  15  59]
 [ 57  20   4   5   8   0   4   3 874  25]
 [ 69  73   7   4   4   0   2   3  28 810]]

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.64      0.64      1000
           1       0.83      0.80      0.82      1000
           2       0.54      0.58      0.56      1000
           3       0.55      0.32      0.40      1000
           4       0.55      0.67      0.61      1000
           5       0.69      0.36      0.47      1000
           6       0.66      0.81      0.73      1000
           7       0.77      0.67     

# CNN with skip connection for Cifar-10

In [63]:
model_cnn_skip = cnn_model_with_skip((32, 32, 3), 4, 4, 32,0,2,1,3)
compilee(model_cnn_skip)
model_cnn_skip.fit(X_train, y_train_one_hot, epochs=5, validation_split=0.2)

Convo Layer 1: KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 32), dtype=tf.float32, name=None), name='activation_25/Relu:0', description="created by layer 'activation_25'")
Convo Layer 2: KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 32), dtype=tf.float32, name=None), name='activation_26/Relu:0', description="created by layer 'activation_26'")
Convo Layer 3: KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 32), dtype=tf.float32, name=None), name='activation_27/Relu:0', description="created by layer 'activation_27'")
Convo Layer 4: KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 32), dtype=tf.float32, name=None), name='activation_28/Relu:0', description="created by layer 'activation_28'")
Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_22 (InputLayer)       [(None, 32, 32, 3)]          0 

In [64]:
print_eval(model_cnn_skip)

313/313 [==============================] - 3s 10ms/step
Confusion Matrix:
[[621  26  77  70  25  14   6  37  67  57]
 [ 22 711  17  25   3  12   9  15  20 166]
 [ 58   6 439 145  68 159  43  63   8  11]
 [  9   6  48 582  31 236  18  58   5   7]
 [ 19   0  89 150 460  98  40 132  10   2]
 [  4   3  36 201  34 643   5  63   4   7]
 [  2   3  44 210  70  76 552  25   4  14]
 [ 10   0  25  61  43 113   4 733   3   8]
 [106  61  18  33  10  22   4  13 666  67]
 [ 12  68   6  41   6  20   8  45  22 772]]

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.62      0.67      1000
           1       0.80      0.71      0.75      1000
           2       0.55      0.44      0.49      1000
           3       0.38      0.58      0.46      1000
           4       0.61      0.46      0.53      1000
           5       0.46      0.64      0.54      1000
           6       0.80      0.55      0.65      1000
           7       0.62      0.73     

# Batch Generator

In [81]:
def data_generator(data, labels, batch_size):
    num_samples = len(data)
    while True:
        indices = np.random.randint(0, num_samples, size=batch_size)
        batch_data = data[indices]
        batch_labels = labels[indices]
        yield batch_data, batch_labels

In [77]:
model_cnn_skip = cnn_model_with_skip((32, 32, 3), 4, 4, 32,0,2,1,3)

Convo Layer 1: KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 32), dtype=tf.float32, name=None), name='activation_33/Relu:0', description="created by layer 'activation_33'")
Convo Layer 2: KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 32), dtype=tf.float32, name=None), name='activation_34/Relu:0', description="created by layer 'activation_34'")
Convo Layer 3: KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 32), dtype=tf.float32, name=None), name='activation_35/Relu:0', description="created by layer 'activation_35'")
Convo Layer 4: KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 32), dtype=tf.float32, name=None), name='activation_36/Relu:0', description="created by layer 'activation_36'")


In [82]:
batch_size = 64
epochs = 5

train_data_generator = data_generator(X_train, y_train_one_hot, batch_size)
validation_data_generator = data_generator(X_test, y_test_one_hot, batch_size)

model_cnn_skip.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

model_cnn_skip.fit_generator(generator=train_data_generator,
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=epochs,
                    validation_data=validation_data_generator,
                    validation_steps=len(X_test) // batch_size)

Epoch 1/5


C:\Users\Harshneel\AppData\Local\Temp\ipykernel_13924\436927477.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_cnn_skip.fit_generator(generator=train_data_generator,
c:\Users\Harshneel\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\backend.py:5577: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


781/781 [==============================] - 120s 152ms/step - loss: 1.7517 - accuracy: 0.3568 - val_loss: 1.4688 - val_accuracy: 0.4542
Epoch 2/5
781/781 [==============================] - 120s 153ms/step - loss: 1.3030 - accuracy: 0.5305 - val_loss: 1.2705 - val_accuracy: 0.5432
Epoch 3/5
781/781 [==============================] - 118s 151ms/step - loss: 1.1290 - accuracy: 0.5928 - val_loss: 1.1624 - val_accuracy: 0.5792
Epoch 4/5
781/781 [==============================] - 118s 151ms/step - loss: 1.0277 - accuracy: 0.6305 - val_loss: 1.2449 - val_accuracy: 0.5678
Epoch 5/5
781/781 [==============================] - 117s 150ms/step - loss: 0.9441 - accuracy: 0.6624 - val_loss: 1.1116 - val_accuracy: 0.6122


# UNET Architecture

In [89]:
def train_initialize():
    train_image  = os.listdir(r'C:\Users\Harshneel\Desktop\DAIICT Work\DL Assignments\Cryo\Cryo\train\tissue images')
    test_image =  os.listdir(r'C:\Users\Harshneel\Desktop\DAIICT Work\DL Assignments\Cryo\Cryo\test\tissue images')
    y_train_image  = os.listdir(r'C:\Users\Harshneel\Desktop\DAIICT Work\DL Assignments\Cryo\Cryo\train\mask binary')
    y_test_image =  os.listdir(r'C:\Users\Harshneel\Desktop\DAIICT Work\DL Assignments\Cryo\Cryo\test\mask binary')


    X_train=[]
    y_train=[]
    for i in range(len(train_image)):
        X_train.append(cv2.imread('C:/Users/Harshneel/Desktop/DAIICT Work/DL Assignments/Cryo/Cryo/train/tissue images/'+train_image[i]))
        y_train.append(cv2.imread('C:/Users/Harshneel/Desktop/DAIICT Work/DL Assignments/Cryo/Cryo/train/mask binary/'+y_train_image[i],cv2.IMREAD_GRAYSCALE))
    y_train = np.array(y_train)
    X_train = np.array(X_train)
    
    X_test=[]
    y_test=[]
    for i in range(len(test_image)):
        X_test.append(cv2.imread('C:/Users/Harshneel/Desktop/DAIICT Work/DL Assignments/Cryo/Cryo/test/tissue images/'+test_image[i]))
        y_test.append(cv2.imread('C:/Users/Harshneel/Desktop/DAIICT Work/DL Assignments/Cryo/Cryo/test/mask binary/'+y_test_image[i],cv2.IMREAD_GRAYSCALE))
    y_test = np.array(y_test)
    X_test = np.array(X_test)
    
    X_train=X_train/255
    y_train=y_train/255
    X_test=X_test/255
    y_test=y_test/255
    
    return X_train,y_train,X_test,y_test

In [90]:
X_train_new,y_train_new,X_test_new,y_test_new=train_initialize()

In [150]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice

def dice_loss(y_true, y_pred, smooth=1):
    dice = dice_coefficient(y_true, y_pred, smooth)
    dice_loss = 1 - dice
    return dice_loss

In [151]:
def unet(input_size):
    inputs = Input(input_size)

    # Contracting Path
    conv1 = Conv2D(4, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(4, 3, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(8, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(8, 3, activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)


    # Bottleneck
    conv4 = Conv2D(16, 3, activation='relu', padding='same')(pool2)
    conv4 = Conv2D(16, 3, activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)

    # Expansive Path

    up6 = UpSampling2D(size=(2, 2))(conv4)
    up6 = Conv2D(8, 2, activation='relu', padding='same')(up6)
    merge6 = Concatenate()([conv2, up6])
    conv6 = Conv2D(8, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(8, 3, activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = Conv2D(4, 2, activation='relu', padding='same')(up7)
    merge7 = Concatenate()([conv1, up7])
    conv7 = Conv2D(4, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(4, 3, activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)

    # Output Layer
    output = Conv2D(1, 1, activation='sigmoid')(conv7)

    model = Model(inputs=inputs, outputs=output)

    return model

In [152]:
model = unet((None,None,3))  
model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coefficient])
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv2d_156 (Conv2D)         (None, None, None, 4)        112       ['input_13[0][0]']            
                                                                                                  
 conv2d_157 (Conv2D)         (None, None, None, 4)        148       ['conv2d_156[0][0]']          
                                                                                                  
 batch_normalization_60 (Ba  (None, None, None, 4)        16        ['conv2d_157[0][0]']          
 tchNormalization)                                                                         

In [188]:
history = model.fit(X_train_new, y_train_new, epochs=10,batch_size=10)

Epoch 1/10
2/2 [==============================] - 6s 3s/step - loss: 0.5650 - dice_coefficient: 0.4350
Epoch 2/10
2/2 [==============================] - 6s 3s/step - loss: 0.5384 - dice_coefficient: 0.4616
Epoch 3/10
2/2 [==============================] - 6s 3s/step - loss: 0.5177 - dice_coefficient: 0.4823
Epoch 4/10
2/2 [==============================] - 5s 3s/step - loss: 0.4800 - dice_coefficient: 0.5200
Epoch 5/10
2/2 [==============================] - 6s 3s/step - loss: 0.4564 - dice_coefficient: 0.5436
Epoch 6/10
2/2 [==============================] - 6s 3s/step - loss: 0.4418 - dice_coefficient: 0.5582
Epoch 7/10
2/2 [==============================] - 6s 3s/step - loss: 0.4352 - dice_coefficient: 0.5648
Epoch 8/10
2/2 [==============================] - 6s 3s/step - loss: 0.4294 - dice_coefficient: 0.5706
Epoch 9/10
2/2 [==============================] - 6s 3s/step - loss: 0.4237 - dice_coefficient: 0.5763
Epoch 10/10
2/2 [==============================] - 6s 3s/step - loss: 0.4

In [189]:
y_pred=model.predict(X_test_new)

1/1 [==============================] - 0s 366ms/step


In [190]:
y_pred= tf.squeeze(y_pred)
dice_coefficient(y_test_new,y_pred).numpy()

0.3086526